<a href="https://colab.research.google.com/github/hakdj/teamproject2/blob/master/chs1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import openpyxl
import xlrd

In [31]:
data = pd.read_excel('./goms_feature_prep_1.xlsx', engine='openpyxl') # 각자 경로로 바꾸기

In [32]:
data.isnull().sum()

jobseekAftGradu        0
jobseekAftGradu_str    0
workWithin1yr          0
workWithin1yr_str      0
wishWorkHr             0
wishSalary             0
JobYNaftGradu          0
JobYNaftGradu_str      0
wayTolivexpense        0
wayTolivexpense_str    0
schoolType             0
schoolType_str         0
majorType              0
majorType_str          0
majorCate              0
majorCate_str          0
major                  0
major_str              0
age                    0
internExp              0
toeicScore             0
tospGrade              0
trainingHr             0
jobseekYN              0
certNum                0
mainJobCate            0
mainJobCate_str        0
midJobCate             0
midJobCate_str         0
subJobCate             0
subJobCate_str         0
dtype: int64

In [33]:
# 결측치 0으로 대체
data = data.fillna(0)

In [34]:
data.corr=data.corr()

In [35]:
data.head()

,jobseekAftGradu,jobseekAftGradu_str,workWithin1yr,workWithin1yr_str,wishWorkHr,wishSalary,JobYNaftGradu,JobYNaftGradu_str,wayTolivexpense,wayTolivexpense_str,schoolType,schoolType_str,majorType,majorType_str,majorCate,majorCate_str,major,major_str,age,internExp,toeicScore,tospGrade,trainingHr,jobseekYN,certNum,mainJobCate,mainJobCate_str,midJobCate,midJobCate_str,subJobCate,subJobCate_str
0,2,아니오,2,아니오,0,0,2,아니오,1,일자리에서의 근로 소득,1,2~3년제,1,인문,1,일본어,1,비즈니스일본어과,23,0,0,0,0,1,0,6,미용·여행·숙박·음식·경비·청소직,54,경호·경비직,541,경호·보안 종사자
1,1,예,2,아니오,3,2400,2,아니오,-1,모름/무응답,1,2~3년제,1,인문,1,일본어,2,일본어과,21,0,0,0,240,1,0,0,무직,0,무직,0,무직
2,2,아니오,2,아니오,0,0,2,아니오,3,가족 친지로부터의 지원,1,2~3년제,1,인문,1,일본어,2,일본어과,21,0,800,0,0,0,1,3,교육·법률·사회복지·경찰·소방직 및 군인,21,교육직,214,문리·기술·예능 강사
3,2,아니오,2,아니오,0,0,2,아니오,1,일자리에서의 근로 소득,1,2~3년제,1,인문,1,일본어,3,관광일어과,25,0,0,0,0,0,0,6,미용·여행·숙박·음식·경비·청소직,52,여행·숙박·오락 서비스직,522,항공기·선박·열차 객실승무원
4,2,아니오,2,아니오,0,0,2,아니오,1,일자리에서의 근로 소득,1,2~3년제,1,인문,1,일본어,3,관광일어과,21,0,0,0,0,1,1,7,영업·판매·운전·운송직,61,영업·판매직,615,판매 종사자


우선 str 다빼고 정확도 추세 보려고 시도함 

In [41]:
data_main = data.drop(['schoolType_str','subJobCate', 'mainJobCate_str', 'midJobCate', 'midJobCate_str', 'subJobCate_str','jobseekAftGradu_str','workWithin1yr_str','JobYNaftGradu_str','wayTolivexpense_str','majorType_str','majorCate_str','major_str'],axis=1)

In [42]:
data_main

,jobseekAftGradu,workWithin1yr,wishWorkHr,wishSalary,JobYNaftGradu,wayTolivexpense,schoolType,majorType,majorCate,major,age,internExp,toeicScore,tospGrade,trainingHr,jobseekYN,certNum,mainJobCate
0,2,2,0,0,2,1,1,1,1,1,23,0,0,0,0,1,0,6
1,1,2,3,2400,2,-1,1,1,1,2,21,0,0,0,240,1,0,0
2,2,2,0,0,2,3,1,1,1,2,21,0,800,0,0,0,1,3
3,2,2,0,0,2,1,1,1,1,3,25,0,0,0,0,0,0,6
4,2,2,0,0,2,1,1,1,1,3,21,0,0,0,0,1,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18158,2,2,0,0,1,-1,2,7,181,1968,27,0,0,0,0,1,0,0
18159,2,2,0,0,2,1,2,7,181,2912,25,0,0,0,0,0,0,3
18160,2,2,0,0,1,1,2,7,181,1361,26,0,0,0,0,0,0,7
18161,2,2,2,3000,2,-1,1,3,20,93,28,0,0,0,0,0,0,0


train,test

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier


from sklearn.ensemble import RandomForestClassifier # 분류는 사용 필요 x
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import plot_importance

### 오차율(표준편차 제곱) 및 score

major_main

In [44]:
X_ohe1 = pd.get_dummies(data_main, columns=['major'])

y_main = X_ohe1['mainJobCate']
x_main = X_ohe1.drop(['mainJobCate'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x_main, y_main, test_size=0.3, random_state=0)
dtr = DecisionTreeRegressor(max_depth=3, random_state=12)
dtr.fit(X_train, y_train)

rfr = RandomForestRegressor(max_depth = 3,random_state=12)
rfr.fit(X_train, y_train)

xgbr = XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=12)
xgbr.fit(X_train, y_train)



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=12, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [45]:
x_main

,jobseekAftGradu,workWithin1yr,wishWorkHr,wishSalary,JobYNaftGradu,wayTolivexpense,schoolType,majorType,majorCate,age,internExp,toeicScore,tospGrade,trainingHr,jobseekYN,certNum,major_1,major_2,major_3,major_4,major_5,major_6,major_7,major_8,major_9,major_10,major_11,major_12,major_13,major_14,major_15,major_16,major_17,major_18,major_19,major_20,major_21,major_22,major_23,major_24,...,major_3357,major_3358,major_3359,major_3360,major_3361,major_3362,major_3363,major_3364,major_3365,major_3366,major_3367,major_3368,major_3369,major_3370,major_3371,major_3372,major_3373,major_3374,major_3375,major_3376,major_3377,major_3378,major_3379,major_3380,major_3381,major_3382,major_3383,major_3384,major_3385,major_3386,major_3387,major_3388,major_3389,major_3390,major_3391,major_3392,major_3393,major_3394,major_3395,major_3396
0,2,2,0,0,2,1,1,1,1,23,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,3,2400,2,-1,1,1,1,21,0,0,0,240,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2,0,0,2,3,1,1,1,21,0,800,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,2,0,0,2,1,1,1,1,25,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,2,0,0,2,1,1,1,1,21,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18158,2,2,0,0,1,-1,2,7,181,27,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18159,2,2,0,0,2,1,2,7,181,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18160,2,2,0,0,1,1,2,7,181,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18161,2,2,2,3000,2,-1,1,3,20,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


DecisionTree

In [46]:
y_train_pred = dtr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)

y_test_pred = dtr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)

Train MSE: 4.1280
Test MSE: 4.2456


In [47]:
dt=DecisionTreeClassifier(random_state=0)
dt.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [48]:
pred=dt.predict(X_test)
print("정확도: {0:.3f}".format(dt.score(X_test,y_test)))

정확도: 0.589


RandomForest

In [49]:
y_train_pred = rfr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)

y_test_pred = rfr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)

y_train_pred = rfr.predict(X_train)

Train MSE: 4.0818
Test MSE: 4.2018


In [50]:
rf=RandomForestClassifier(random_state=0, max_depth=4)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [51]:
pred=rf.predict(X_test)
print("정확도:{0:.3f}".format(accuracy_score(y_test,pred)))

정확도:0.327


XGB

In [52]:

y_train_pred = xgbr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)

y_test_pred = xgbr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)



Train MSE: 3.8087
Test MSE: 4.0632


In [54]:
xgb=XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=4)
xgb.fit(X_train,y_train)
xgb_pred=xgb.predict(X_test)


In [56]:
accuracy_score(y_test,xgb_pred)
print("정확도:{0:.3f}".format(accuracy_score(y_test,xgb_pred)))

정확도:0.671


major_mid

In [ ]:
X_ohe1 = pd.get_dummies(data_main, columns=['major'])
y_main = X_ohe1['midJobCate']
x_main = X_ohe1.drop(['midJobCate'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x_main, y_main, test_size=0.3, random_state=0)
dtr = DecisionTreeRegressor(max_depth=3, random_state=12)
dtr.fit(X_train, y_train)

rfr = RandomForestRegressor(max_depth = 3,random_state=12)
rfr.fit(X_train, y_train)

xgbr = XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=12)
xgbr.fit(X_train, y_train)


In [ ]:
x_main